# Case
**Creates binary labels in raster format**

The labels are either based on a provided text- or Esri-shapefile.<br>
To spatially align the labels with other data sets, a template raster (base raster) can be provided.


In [11]:
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import os

from pathlib import Path
from typing import List, Tuple, Optional, Union
rasterio.enums import MergeAlg
    
from rasterio import features

import sys
if sys.version_info < (3, 9):
    from importlib_resources import files
else:
    from importlib.resources import files

from beak.utilities.conversions import create_geodataframe_from_points, create_geodataframe_from_polygons, _rasterize_vector_process
from beak.utilities.io import save_raster, check_path, load_raster, load_dataset


# Load data

**User definitions**

In [12]:
# Paths
BASE_PATH = files("beak.data")
PATH_DATACUBE = BASE_PATH / "LAWLEY22" / "RAW" / "2021_Table04_Datacube.csv"
PATH_SHAPEFILE = BASE_PATH / "CRITICAL_MINERAL_DEPOSITS" / "critical_mineral_deposits.shp"
PATH_BASE_RASTER = BASE_PATH / "BASE_RASTERS" / "EPSG_4326_RES_0_025_COMPLETE_DATASET.tif"

# Columns for x and y coordinates
LONG_COL = "Longitude_EPSG4326"
LAT_COL = "Latitude_EPSG4326"

# Reference coordinate system code
EPSG_CODE = 4326

# Rows to read from the datacube
NROWS = None

# Filter datacube by region
REGIONS =["United States of America", "Canada"]
COL_REGION_FILTER = "Country_Majority"

# Select relevant columns to be used as labels
LABELS = "Training_MVT_Deposit=='Present' | Training_MVT_Occurrence=='Present'"

# Create Labels

## From textfile or data set

In [5]:
from beak.utilities.conversions import create_binary_raster

# Load data
data = load_dataset(PATH_DATACUBE, nrows=NROWS)
data = data[data[COL_REGION_FILTER].isin(REGIONS)]

# Create geodataframe from points
gdf = create_geodataframe_from_points(data, LONG_COL, LAT_COL, EPSG_CODE)

# Load base raster
base_raster = load_raster(PATH_BASE_RASTER)

# Output path
out_path = "output_from_textfile.tif"

# Save raster and labels array
labels_array = create_binary_raster(gdf, base_raster, query=LABELS, all_touched=False, same_shape=True, fill_negatives=True, out_file=out_path)

## From shapefile

In [21]:
from beak.utilities.conversions import create_binary_raster

# Load shapefile and create geodataframe
gdf = gpd.read_file(PATH_SHAPEFILE)

# Load base raster
base_raster = load_raster(PATH_BASE_RASTER)

# Output path
out_path = "output_from_shapefile.tif"

# Save raster and labels array
labels_array = create_binary_raster(gdf, base_raster, query="DepType == 'S-R-V tungsten'", all_touched=False, same_shape=True, fill_negatives=True, out_file=out_path)